## utils.mem
## Memory management utils
内存管理的实用功能。 目前主要用于GPU。

<b>gpu_mem_get</b>

`gpu_mem_get(id=None)`

获取所有gpu id，已用和可用内存（以MB为单位）。 如果未传递id，则使用当前选择的torch设备

`gpu_mem_get`
* for gpu returns GPUMemory(total, free, used)
* for cpu returns GPUMemory(0, 0, 0)
* for invalid gpu id returns GPUMemory(0, 0, 0)

In [1]:
from fastai.utils.mem import *

In [2]:
gpu_mem_get()

GPUMemory(total=4096, free=3934, used=161)

<b>gpu_mem_get_all</b>

`gpu_mem_get_all()`

获取每个可用gpu的总内存，已用内存和可用内存（以MB为单位）

`gpu_mem_get_all`

for gpu returns [ GPUMemory(total_0, free_0, used_0), GPUMemory(total_1, free_1, used_1), .... ]
for cpu returns []

In [4]:
gpu_mem_get_all()

[GPUMemory(total=4096, free=3934, used=161)]

<b>gpu_mem_get_free</b>

`gpu_mem_get_free()`

获取当前所选gpu id的空闲内存（以MB为单位），无需清空缓存

In [6]:
gpu_mem_get_free()

3934

<b>gpu_mem_get_free_no_cache</b>

`gpu_mem_get_free_no_cache()`

在清空缓存后，获取当前所选gpu id的空闲内存（以MB为单位）

In [8]:
gpu_mem_get_free_no_cache()

3934

<b>gpu_mem_get_used</b>

`gpu_mem_get_used()`

获取当前所选gpu id的内存（以MB为单位），不用清空缓存

In [10]:
gpu_mem_get_used()

161

<b>gpu_mem_get_used_no_cache</b>

`gpu_mem_get_used_no_cache()`

在清空缓存后，获取当前所选gpu id的内存（以MB为单位）

In [11]:
gpu_mem_get_used_no_cache()

161

<b>gpu_mem_get_used_fast</b>

`gpu_mem_get_used_fast(gpu_handle)`

获取当前所选gpu id的内存（以MB为单位），不清空缓存，需要gpu_handle arg

<b>gpu_with_max_free_mem</b>

`gpu_with_max_free_mem()`

为具有最高可用RAM的第一个gpu获取[gpu_id，its_free_ram]

gpu_with_max_free_mem:

* for gpu returns: gpu_with_max_free_ram_id, its_free_ram
* for cpu returns: None, 0

In [14]:
gpu_with_max_free_mem()

(0, 3934)

<b>preload_pytorch</b>

`preload_pytorch()`

preload_pytorch在测量GPU内存时很有用，因为第一次对cuda进行任何操作都是由pytorch执行的，通常大约0.5GB被CUDA上下文使用。



In [15]:
preload_pytorch()

<b>class GPUMemory</b>

`GPUMemory(total, free, used) :: tuple`

GPUMemory是一个由gpu_mem_get和gpu_mem_get_all等函数返回的namedtuple。

<b>b2mb</b>

`b2mb(num)`

将Bs转换为MB并向下舍入

b2mb是一个帮助实用程序，只执行int（bytes / 2 ** 20）

## Memory Tracing Utils
<b>class GPUMemTrace</b>

`GPUMemTrace(silent=False, ctx=None, on_exit_report=True)`

跟踪分配和峰值GPU内存使用（增量）。

参数：

* silent：生成报告和report_n_reset的快捷方式，无需删除这些调用 - 这可以从构造函数中完成，或者您可以在任何地方调用静默方法来执行相同操作。
* ctx：报告中的默认上下文注释
* on_exit_report：ctx manager exit上的自动报告（默认为True）

定义：

* Delta Used是当前使用的内存与计数器开始时使用的内存之间的差异。

* Delta Peaked是内存开销，如果有的话。 它分两步计算：

    1. 基本测量值是计数器开始时峰值存储器和已用存储器之间的差值。
    2. 然后，如果使用的delta为正，则从基值中减去。

        它表示blip的大小。

        警告：目前高峰内存使用情况跟踪是使用python线程实现的，这是非常不可靠的，因为无法保证线程在峰值内存发生时有机会运行（或者它可能没有机会 一直跑。） 因此我们需要pytorch来实现多个并发和可重置的torch.cuda.max_memory_allocated计数器。 请投票支持此功能请求。
        
用法示例：

Setup：

In [16]:
from fastai.utils.mem import GPUMemTrace
def some_code(): pass
mtrace = GPUMemTrace()

示例1：通过报告（打印）和通过数据（返回）访问器进行的基本测量

In [17]:
some_code()
mtrace.report()
delta_used, delta_peaked = mtrace.data()

some_code()
mtrace.report('2nd run of some_code()')
delta_used, delta_peaked = mtrace.data()

△Used Peaked MB:      0      0
△Used Peaked MB:      0      0 (2nd run of some_code())


如果您有许多报表调用，并且想要了解哪个是输出中的哪个，那么report的可选subctx参数会很有用。

示例2：循环测量，在每次运行之前重置计数器

In [18]:
for i in range(10):
    mtrace.reset()
    some_code()
    mtrace.report(f'i={i}')

△Used Peaked MB:      0      0 (i=0)
△Used Peaked MB:      0      0 (i=1)
△Used Peaked MB:      0      0 (i=2)
△Used Peaked MB:      0      0 (i=3)
△Used Peaked MB:      0      0 (i=4)
△Used Peaked MB:      0      0 (i=5)
△Used Peaked MB:      0      0 (i=6)
△Used Peaked MB:      0      0 (i=7)
△Used Peaked MB:      0      0 (i=8)
△Used Peaked MB:      0      0 (i=9)


reset重置所有计数器。

示例3：与示例2类似，但报告会自动重置计数器

In [20]:
mtrace.reset()
for i in range(10):
    some_code()
    mtrace.report_n_reset(f'i={i}')

△Used Peaked MB:      0      0 (i=0)
△Used Peaked MB:      0      0 (i=1)
△Used Peaked MB:      0      0 (i=2)
△Used Peaked MB:      0      0 (i=3)
△Used Peaked MB:      0      0 (i=4)
△Used Peaked MB:      0      0 (i=5)
△Used Peaked MB:      0      0 (i=6)
△Used Peaked MB:      0      0 (i=7)
△Used Peaked MB:      0      0 (i=8)
△Used Peaked MB:      0      0 (i=9)


创建GPUMemTrace对象后立即开始跟踪，并在删除该对象时停止。 但它也可以停止，也可以手动启动。

In [21]:
mtrace.start()
mtrace.stop()

如果要冻结GPUMemTrace对象并且能够在某个时间停止查询其数据，则stop特别有用。

<b>报告：</b>

在报表中，您可以打印通过构造函数传递的主要上下文：

In [22]:
mtrace = GPUMemTrace(ctx="foobar")
mtrace.report()

△Used Peaked MB:      0      0 (foobar)


然后根据需要添加子上下文注释：

In [23]:
mtrace = GPUMemTrace(ctx="foobar")
mtrace.report('1st try')
mtrace.report('2nd try')

△Used Peaked MB:      0      0 (foobar: 1st try)
△Used Peaked MB:      0      0 (foobar: 2nd try)


上下文和子上下文都是可选的，如果您在代码周围的不同位置放置GPUMemTrace，它们非常有用。

您可以静默报告调用，无需通过构造函数或静默删除它们：

In [29]:
mtrace = GPUMemTrace(silent=True)
mtrace.report() # nothing will be printed
mtrace.silent = False
mtrace.report() # printing resumed
mtrace.silent = True
mtrace.report() # nothing will be printed

△Used Peaked MB:      0      0


<b>Context Manager:</b>

GPUMemTrace也可以用作上下文管理器：

自动报告已使用和已达到峰值的增量：

In [31]:
with GPUMemTrace(): some_code()

△Used Peaked MB:      0      0 (exit)


如果您想添加上下文：

In [32]:
with GPUMemTrace(ctx='some context'): some_code()

△Used Peaked MB:      0      0 (some context: exit)


上下文管理器使用子上下文退出来指示在退出上下文之后报告。

报告是自动完成的，由于返回调用，这在函数中特别有用：

In [34]:
def some_func():
    with GPUMemTrace(ctx='some_func'):
        # some code
        return 1
some_func()

△Used Peaked MB:      0      0 (some_func: exit)


1

所以尽管调用，你仍然可以获得完美的报告。 ctx对于指定上下文非常有用，以防您通过代码进行了许多调用，并且想知道哪个是哪个。

当然，除了执行上述操作之外，您还可以使用gpu_mem_trace装饰器自动执行此操作，包括使用函数或方法名称作为上下文。 因此，下面的示例在不修改函数的情况下也是如此。

In [35]:
@gpu_mem_trace
def some_func():
    # some code
    return 1
some_func()

△Used Peaked MB:      0      0 (some_func: exit)


1

如果您不希望自动报告，只需在构造函数中传递on_exit_report = False：

In [36]:
with GPUMemTrace(ctx='some_func', on_exit_report=False) as mtrace:
    some_code()
mtrace.report("measured in ctx")

△Used Peaked MB:      0      0 (some_func: measured in ctx)


或与上下文注释相同：

In [37]:
with GPUMemTrace(on_exit_report=False) as mtrace: some_code()
print(mtrace) # or mtrace.report()

△Used Peaked MB:      0      0


当然，您可以获得数值数据（以舍入的MB为单位）：

In [38]:
with GPUMemTrace() as mtrace: some_code()
delta_used, delta_peaked = mtrace.data()

△Used Peaked MB:      0      0 (exit)


<b>gpu_mem_trace</b>

`gpu_mem_trace(func)`

一个装饰器，运行GPUMemTrace w/ 报告方法

这允许您使用以下内容修饰任何函数或方法：

In [39]:
@gpu_mem_trace
def my_function(): pass
# run:
my_function()

△Used Peaked MB:      0      0 (my_function: exit)


它将自动打印包含函数名称的报告作为上下文